In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
print(tf.__version__)

2.2.0-rc1


In [0]:
!pip install transformers

     |████████████████████████████████| 542kB 2.7MB/s 
     |████████████████████████████████| 3.7MB 15.0MB/s 
     |████████████████████████████████| 1.0MB 27.2MB/s 
     |████████████████████████████████| 870kB 45.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=1063da7db8a0cb41e525b5dc810ba6f2befcbda6a72dd83b7754199b7e52a9d7
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import re

Так как в определении значения большую роль играет контекст, применим две модификации Bert: Albert и RoBERTa.

## Albert

Применим предобученные модели base и large

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DeepLearning/Bert/twitter/train.csv', encoding='utf-8')

Let's extract the sentences and labels of our training set as numpy ``ndarrays``.

In [0]:
# Get the lists of sentences and their labels.
sentences = df['text'].values
labels = df['target'].values

### Albert base

In [0]:
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2", do_lower_case=True)

Let's apply the tokenizer to one sentence just to see the output.

In [0]:
print(' Original: ', sentences[0])

print('Tokenized: ', tokenizer.tokenize(sentences[0]))

print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Tokenized:  ['▁our', '▁deeds', '▁are', '▁the', '▁reason', '▁of', '▁this', '▁#', 'earth', 'qua', 'ke', '▁may', '▁', 'allah', '▁forgive', '▁us', '▁all']
Token IDs:  [318, 18055, 50, 14, 1215, 16, 48, 6926, 12322, 5418, 1048, 123, 13, 12209, 8591, 182, 65]


When we actually convert all of our sentences, we'll use the ``tokenize.encode`` function to handle both steps, rather than calling tokenize and ``convert_tokens_to_ids`` separately.

Before we can do that, though, we need to talk about some of BERT's formatting requirements.

In [0]:

input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,
                        add_special_tokens = True,
                  )
    
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Token IDs: [2, 318, 18055, 50, 14, 1215, 16, 48, 6926, 12322, 5418, 1048, 123, 13, 12209, 8591, 182, 65, 3]


In [0]:
print('Max sentence length: ', max([len(sen) for sen in input_ids]))

Max sentence length:  100


In [0]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 85

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 85 values...

Padding token: "<pad>", ID: 0

Done.


In [0]:

attention_masks = []

for sent in input_ids:
    
    att_mask = [int(token_id > 0) for token_id in sent]
    
    attention_masks.append(att_mask)

In [0]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [0]:
BATCH_SIZE = 32

In [0]:
def train_gen():
  for input_id, attention_mask, label_id in zip(train_inputs, train_masks, train_labels ):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
        }, label_id
    )

def validation_gen():
  for input_id, attention_mask, label_id in zip(validation_inputs, validation_masks, validation_labels):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
        }, label_id
    )


train_data = tf.data.Dataset.from_generator(
    train_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

validation_data = tf.data.Dataset.from_generator(
    validation_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

In [0]:
train_data = train_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


In [0]:
validation_data = validation_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


In [0]:
from transformers import TFAlbertForSequenceClassification, AdamW, BertConfig

model = TFAlbertForSequenceClassification.from_pretrained(
    "albert-base-v2",
    num_labels = 2, 
    output_attentions = False,
    output_hidden_states = False,
)

In [0]:
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_42 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 11,685,122
Trainable params: 11,685,122
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

In [0]:
if len(np.unique(labels)) > 1:
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [0]:
train_examples = len(train_inputs)
valid_examples = len(validation_inputs)

print('In training set: {}'.format(train_examples))
print('In validation set: {}'.format(valid_examples))
# Train and evaluate using tf.keras.Model.fit()
train_steps = train_examples // BATCH_SIZE
valid_steps = valid_examples // BATCH_SIZE

print('Count of steps:\n\tTraining set: {} \n\tValidation set: {}'.format(train_steps, valid_steps))

In training set: 6851
In validation set: 762
Count of steps:
	Training set: 214 
	Validation set: 23


In [0]:
EPOCHS = 10

In [0]:
history = model.fit(
    train_data,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=validation_data,
    validation_steps=valid_steps
)

Train for 214 steps, validate for 23 steps
Epoch 1/10
214/214 [==============================] - 76s 354ms/step - loss: 0.2556 - accuracy: 0.8994 - val_loss: 0.3968 - val_accuracy: 0.8451
Epoch 2/10
214/214 [==============================] - 75s 351ms/step - loss: 0.2010 - accuracy: 0.9267 - val_loss: 0.4489 - val_accuracy: 0.8383
Epoch 3/10
214/214 [==============================] - 75s 353ms/step - loss: 0.1902 - accuracy: 0.9280 - val_loss: 0.5664 - val_accuracy: 0.8125
Epoch 4/10
214/214 [==============================] - 75s 352ms/step - loss: 0.1730 - accuracy: 0.9359 - val_loss: 0.5491 - val_accuracy: 0.8410
Epoch 5/10
214/214 [==============================] - 75s 352ms/step - loss: 0.1454 - accuracy: 0.9479 - val_loss: 0.5271 - val_accuracy: 0.8465
Epoch 6/10
214/214 [==============================] - 76s 353ms/step - loss: 0.1094 - accuracy: 0.9614 - val_loss: 0.5866 - val_accuracy: 0.8397
Epoch 7/10
214/214 [==============================] - 75s 353ms/step - loss: 0.0898 - a

Фактическое количество эпох = 20, после 10 эпохи (на валидации = 0.8465) качество не улучшается. Предсказание меток сделано после 10 эпох.

In [0]:
 model.evaluate(validation_data, steps=valid_steps)

23/23 [==============================] - 4s 191ms/step - loss: 0.4050 - accuracy: 0.8410


[0.40503597259521484, 0.8410326242446899]

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/test_tweets.csv', encoding='utf-8')

In [0]:
sentences = test['text'].values

In [0]:
input_ids = []

for sent in sentences:
    encoded_sent =  tf.constant(tokenizer.encode(
                        sent,                     
                        add_special_tokens = True,
                  ))[None, :]
    
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Just happened a terrible car crash
Token IDs: tf.Tensor([[   2  114 1190   21 5803  349 4597    3]], shape=(1, 8), dtype=int32)


In [0]:
outputs = []
n=0
for i in input_ids:
    n+=1
    outputs.append(np.argmax(model(i)))

In [0]:
results = pd.DataFrame()

In [0]:
results['id'] = test['id']
results['target'] = outputs

In [0]:
results.to_csv('/content/gdrive/My Drive/albert_sub.csv',index=False)

### Albert Large

In [0]:
from transformers import AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained("albert-large-v1", do_lower_case=True)

In [0]:
from transformers import TFAlbertForSequenceClassification, AdamW, BertConfig

model = TFAlbertForSequenceClassification.from_pretrained(
    "albert-large-v1",
    num_labels = 2, 
    output_attentions = False,
    output_hidden_states = False,
)

In [0]:
input_ids = []

for sent in sentences:
    encoded_sent = tokenizer.encode(
                        sent,
                        add_special_tokens = True,
                  )
    
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Token IDs: [2, 318, 18055, 50, 14, 1215, 16, 48, 6926, 12322, 5418, 1048, 123, 13, 12209, 8591, 182, 65, 3]


In [0]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 85

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 85 values...

Padding token: "<pad>", ID: 0

Done.


In [0]:
attention_masks = []

for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    
    attention_masks.append(att_mask)

In [0]:
type_masks = []

for sent in input_ids:

    mask = [0]*MAX_LEN
    type_masks.append(mask)

In [0]:
from sklearn.model_selection import train_test_split

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=42, test_size=0.1)
train_type, validation_type, _, _ = train_test_split(type_masks, labels,
                                             random_state=42, test_size=0.1)

In [0]:
BATCH_SIZE = 32

In [0]:
def train_gen():
  for input_id, attention_mask, label_id, token_type_ids in zip(train_inputs, train_masks, train_labels, train_type):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }, label_id
    )

def validation_gen():
  for input_id, attention_mask, label_id, token_type_ids in zip(validation_inputs, validation_masks, validation_labels, validation_type):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }, label_id
    )

train_data = tf.data.Dataset.from_generator(
    train_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
        "token_type_ids": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

validation_data = tf.data.Dataset.from_generator(
    validation_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
        "token_type_ids": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

In [0]:
train_data = train_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


In [0]:
validation_data = validation_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


In [0]:
model.summary()

Model: "tf_albert_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  17683968  
_________________________________________________________________
dropout_11 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2050      
Total params: 17,686,018
Trainable params: 17,686,018
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

In [0]:
if len(np.unique(labels)) > 1:
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [0]:
train_examples = len(train_inputs)
valid_examples = len(validation_inputs)

print('In training set: {}'.format(train_examples))
print('In validation set: {}'.format(valid_examples))
train_steps = train_examples // BATCH_SIZE
valid_steps = valid_examples // BATCH_SIZE

print('Count of steps:\n\tTraining set: {} \n\tValidation set: {}'.format(train_steps, valid_steps))

In training set: 6851
In validation set: 762
Count of steps:
	Training set: 214 
	Validation set: 23


In [0]:
EPOCHS = 10

Так же, как и в первом случае, фактическое количество эпох = 20, максимальное качество получаем на 11, однако оно хуже лучшего по модели base.

In [0]:
model.fit(
    train_data,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=validation_data,
    validation_steps=valid_steps
)

Epoch 1/10
214/214 [==============================] - 221s 1s/step - loss: 0.3909 - accuracy: 0.8375 - val_loss: 0.4368 - val_accuracy: 0.8193
Epoch 2/10
214/214 [==============================] - 221s 1s/step - loss: 0.3599 - accuracy: 0.8554 - val_loss: 0.4664 - val_accuracy: 0.8057
Epoch 3/10
214/214 [==============================] - 220s 1s/step - loss: 0.4410 - accuracy: 0.8086 - val_loss: 0.4755 - val_accuracy: 0.8043
Epoch 4/10
214/214 [==============================] - 221s 1s/step - loss: 0.3540 - accuracy: 0.8556 - val_loss: 0.5024 - val_accuracy: 0.7989
Epoch 5/10
214/214 [==============================] - 221s 1s/step - loss: 0.3221 - accuracy: 0.8711 - val_loss: 0.5024 - val_accuracy: 0.8003
Epoch 6/10
214/214 [==============================] - 221s 1s/step - loss: 0.2982 - accuracy: 0.8836 - val_loss: 0.5142 - val_accuracy: 0.7989
Epoch 7/10
214/214 [==============================] - 221s 1s/step - loss: 0.2773 - accuracy: 0.8984 - val_loss: 0.5285 - val_accuracy: 0.7935

## RoBERTa Large

RoBERTa base дает схожие с Albert large значения качества, поэтому сразу применим модель large.

In [0]:
from transformers import RobertaTokenizer
from transformers import TFRobertaForSequenceClassification

In [0]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', do_lower_case=True)
model = TFRobertaForSequenceClassification.from_pretrained('roberta-large', num_labels = 2, output_attentions = False, output_hidden_states = False)

### tokenization

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DeepLearning/Bert/twitter/train.csv', encoding='utf-8')

In [0]:
sentences = df['text'].values
labels = df['target'].values

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                  )
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sent)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Token IDs: [0, 84, 28519, 32, 5, 1219, 9, 42, 849, 25581, 2253, 5113, 189, 70, 895, 20184, 201, 70, 2]


In [0]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 85

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')


Padding/truncating all sentences to 85 values...

Padding token: "<pad>", ID: 1

Done.


Using TensorFlow backend.


In [0]:
# Create attention masks
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [0]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split

# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=42, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=42, test_size=0.1)

In [0]:
BATCH_SIZE = 32

In [0]:
def train_gen():
  for input_id, attention_mask, label_id in zip(train_inputs, train_masks, train_labels ):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
        }, label_id
    )

def validation_gen():
  for input_id, attention_mask, label_id in zip(validation_inputs, validation_masks, validation_labels):
    yield(
        {                        
            "input_ids": input_id,
            "attention_mask": attention_mask,
        }, label_id
    )


train_data = tf.data.Dataset.from_generator(
    train_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

validation_data = tf.data.Dataset.from_generator(
    validation_gen, 
    ({"input_ids": tf.int32, "attention_mask": tf.int32}, tf.int64 ),
    ({ 
        "input_ids": tf.TensorShape([None]),
        "attention_mask": tf.TensorShape([None]),
      },
    tf.TensorShape([]),)
)

In [0]:
train_data = train_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


In [0]:
validation_data = validation_data.shuffle(64).batch(BATCH_SIZE).repeat(-1)


### model

In [0]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  355359744 
_________________________________________________________________
classifier (TFRobertaClassif multiple                  1051650   
Total params: 356,411,394
Trainable params: 356,411,394
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)

In [0]:
if len(np.unique(labels)) > 1:
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [0]:
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [0]:
train_examples = len(train_inputs)
valid_examples = len(validation_inputs)

print('In training set: {}'.format(train_examples))
print('In validation set: {}'.format(valid_examples))
# Train and evaluate using tf.keras.Model.fit()
train_steps = train_examples // BATCH_SIZE
valid_steps = valid_examples // BATCH_SIZE

print('Count of steps:\n\tTraining set: {} \n\tValidation set: {}'.format(train_steps, valid_steps))

In training set: 6851
In validation set: 762
Count of steps:
	Training set: 214 
	Validation set: 23


In [0]:
EPOCHS = 2

In [0]:
model.fit(
    train_data,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_data=validation_data,
    validation_steps=valid_steps
)

Epoch 1/2
214/214 [==============================] - 233s 1s/step - loss: 0.4798 - accuracy: 0.7821 - val_loss: 0.3921 - val_accuracy: 0.8383
Epoch 2/2
214/214 [==============================] - 228s 1s/step - loss: 0.3896 - accuracy: 0.8403 - val_loss: 0.3849 - val_accuracy: 0.8451


После 2 эпохи качество падает до 0.6 и не поднимается выше полученного на второй. Предскажем значения для тестовой выборки.

In [0]:
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/DeepLearning/Bert/twitter/test.csv', encoding='utf-8')

In [0]:
sentences = test['text'].values

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

for sent in sentences:
    encoded_sent =  tf.constant(tokenizer.encode(
                        sent,                     
                        add_special_tokens = True,
                  ))[None, :]
    
    input_ids.append(encoded_sent)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  Just happened a terrible car crash
Token IDs: tf.Tensor([[   0   95 1102   10 6587  512 2058    2]], shape=(1, 8), dtype=int32)


In [0]:
outputs = []
n=0
for i in input_ids:
    n+=1
    print(n, end='-')
    outputs.append(np.argmax(model(i)))

1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99-100-101-102-103-104-105-106-107-108-109-110-111-112-113-114-115-116-117-118-119-120-121-122-123-124-125-126-127-128-129-130-131-132-133-134-135-136-137-138-139-140-141-142-143-144-145-146-147-148-149-150-151-152-153-154-155-156-157-158-159-160-161-162-163-164-165-166-167-168-169-170-171-172-173-174-175-176-177-178-179-180-181-182-183-184-185-186-187-188-189-190-191-192-193-194-195-196-197-198-199-200-201-202-203-204-205-206-207-208-209-210-211-212-213-214-215-216-217-218-219-220-221-222-223-224-225-226-227-228-229-230-231-232-233-234-235-236-237-238-239-240-241-242-243-244-245-246-247-248-249-250-251-252-253-254-255-256-257-258-259-260-261-262-263-264-265-266-267-268-269-270-271-272-273-274-275-276-277-

In [0]:
results = pd.DataFrame()

In [0]:
results['id'] = test['id']
results['target'] = outputs

In [0]:
results.to_csv('/content/gdrive/My Drive/Colab Notebooks/DeepLearning/Bert/robertalarge2.csv',index=False)

Оценка на тесте для RoBERTa оказывается выше Albert (0.81799 vs 0.79856)